# **ANÁLISIS DE RESEÑAS DE HOTELES (TRIPADVISOR)**

# PASO 1: INSTALACIÓN DE LIBRERÍAS NECESARIAS

In [ ]:
!pip install -q pandas numpy matplotlib seaborn nltk spacy scikit-learn sentence-transformers deep-translator transformers bertopic umap-learn sentence-transformers plotly==6.0.1

# Explicación de librerías:
# nltk               → Natural Language Toolkit para procesamiento de texto
# bertopic           → Para modelado de tópicos avanzado con transformers
# spacy              → Para procesamiento de lenguaje natural en español
# scikit-learn       → Para algoritmos de machine learning y vectorización
# umap-learn         → Para reducción de dimensionalidad en BERTopic
# sentence-transformers → Para embeddings semánticos
# deep-translator    → Para traducir texto usando Google Translate
# transformers       → Para clasificación de texto con BERT

# Descargar el modelo en español para spaCy
# Este modelo contiene datos pre-entrenados para análisis lingüístico en español
!python -m spacy download es_core_news_sm

# PASO 2: IMPORTAR LIBRERÍAS

In [ ]:
import pandas as pd                    # Para trabajar con DataFrames
import numpy as np                     # Para operaciones numéricas
import matplotlib.pyplot as plt        # Para crear gráficos estáticos
import seaborn as sns                  # Para gráficos estadísticos avanzados
import plotly.io as pio                  # Para configurar y exportar gráficos de Plotly
from bertopic import BERTopic            # Para modelado de temas
from sklearn.cluster import KMeans       # Para clustering (en este ejemplo, KMeans)
from umap import UMAP                    # Para reducción de dimensionalidad
import re                             # Para expresiones regulares
from google.colab import files        # Para subir archivos en Colab
from deep_translator import GoogleTranslator  # Para traducir texto usando Google Translate
import time                           # Para pausas entre traducciones
from transformers import pipeline     # Aplicamos librería de transformers para clasificación automática de texto
from tqdm import tqdm                 # Usar tqdm para mostrar progreso del análisis
import warnings # Para controlar mensajes de advertencia
warnings.filterwarnings('ignore', category=SyntaxWarning)

# Importar bibliotecas para análisis de texto avanzado
import spacy                          # Para procesamiento de lenguaje natural
import nltk                           # Para tokenización y stopwords
from nltk.sentiment import SentimentIntensityAnalyzer # Importar VADER desde NLTK
from nltk.corpus import stopwords

# Configuraciones NLTK
nltk.download('stopwords', quiet=True)
nltk.download('vader_lexicon', quiet=True)

# Obtener stopwords en español
stop_words = set(stopwords.words('spanish'))
# Añadir palabras específicas que queremos excluir en reseñas de hoteles
stop_words.update(["cabo", "poder", "hacer", "sido", "haber", "tener", "día", "él"])

# Cargar el modelo de spaCy para español
print("Cargando modelo de lenguaje spaCy para español...")
try:
    nlp = spacy.load("es_core_news_sm")
except:
    print("Instalando modelo de spaCy (primera ejecución)...")
    import spacy.cli
    spacy.cli.download("es_core_news_sm")
    nlp = spacy.load("es_core_news_sm")

# Descargar el lexicon de VADER (análisis de sentimientos)
try:
    nltk.data.find('vader_lexicon')
except LookupError:
    print("Descargando lexicon de VADER...")
    nltk.download('vader_lexicon', quiet=True)

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Crear colormap personalizado con los colores de sentimientos
from matplotlib.colors import ListedColormap
import matplotlib.colors as mcolors

# PASO 3: SUBIR EL ARCHIVO EXCEL A GOOGLE COLAB

In [ ]:
print("\n📁 Haz clic en el botón 'Elegir archivos' para subir tu archivo Excel con las reseñas")
archivos_subidos = files.upload()  # Abrir el selector de archivos

# PASO 4: CARGAR Y EXPLORAR LOS DATOS

In [ ]:
# Capturar el nombre del archivo subido
excel_filename = list(archivos_subidos.keys())[0]
print(f"▶️ Usando: {excel_filename}")  # Mostrar el nombre del archivo

# Cargar el archivo Excel
df = pd.read_excel(excel_filename)
print(f"✅ Archivo cargado: {df.shape[0]} filas y {df.shape[1]} columnas")

# Mostrar información básica
print("\n=== INFORMACIÓN DEL DATASET ===")
print(f"Total de reseñas: {len(df)}")
print(f"\nColumnas disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

# Vista previa de los datos
print("\n🔍 Vista previa de los datos:")
display(df.head())

# PASO 5: LIMPIAR Y PREPARAR LOS DATOS

## PASO 5.1: Función para extraer calificación numérica

In [ ]:
def extraer_calificacion(row):
    """
    Une las tres columnas de calificación y extrae el valor numérico
    Ejemplo: '5,0 de 5 burbujas' → 5.0
    """

    # Lista de las tres columnas de calificación
    calificaciones = [row['Calificacion'], row['Calificacion2'], row['Calificacion3']]

    # Buscar la primera calificación que no sea NaN
    for cal in calificaciones:
        if pd.notna(cal):
            # Buscar el patrón 'X,X de 5 burbujas' en el texto
            match = re.search(r'(\d+),(\d+)', str(cal))
            if match:
                # Convertir a decimal: '5,0' → 5.0
                return float(f"{match.group(1)}.{match.group(2)}")

    return np.nan

## PASO 5.2: Función para convertir fechas del español a datetime

In [ ]:
def convertir_fecha(fecha_str):
    """
    Convierte la fecha del formato español a datetime
    Ejemplo: 'Fecha de la estancia: marzo de 2024' → 2024-03-01

    Explicación del resultado:
    - Toma texto como "Fecha de la estancia: marzo de 2024"
    - Lo convierte a formato datetime estándar: 2024-03-01
    - Usa día 1 como predeterminado para tener fechas consistentes
    - El resultado es tipo datetime64[ns] que permite análisis temporal
    """

    if pd.isna(fecha_str):
        return None

    try:
        # Mapeo de meses en español a números
        meses = {
            'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
            'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
            'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
        }

        # Limpiar el texto y extraer la parte de la fecha
        texto = str(fecha_str).lower()
        if 'fecha de la estancia:' in texto:
            texto = texto.split('fecha de la estancia:')[1].strip()

        # Extraer mes y año (formato: "marzo de 2024")
        partes = texto.split(' de ')
        if len(partes) == 2:
            mes_nombre = partes[0].strip()
            año = int(partes[1].strip())
            mes = meses.get(mes_nombre, 1)
            # Crear fecha usando día 1 para consistencia
            # "marzo de 2024" → 2024-03-01 00:00:00
            return pd.Timestamp(año, mes, 1)

    except:
        return None

    return None

## PASO 5.3: Función para limpiar y categorizar nacionalidad

In [ ]:
def limpiar_nacionalidad(nacionalidad_str):
    """
    Limpia y categoriza la nacionalidad del contribuyente
    Si solo contiene información de contribuciones, devuelve None
    """

    if pd.isna(nacionalidad_str):
        return None

    # Convertir a string y limpiar
    texto = str(nacionalidad_str).strip()

    # Si es solo información de contribuciones (ej: "1 contribución"), devolver None
    if re.match(r'^\d+\s+contribuci[oó]n\s*$', texto):
        return None

    # Remover menciones de contribuciones al inicio si hay más texto después
    texto = re.sub(r'^\d+\s+contribuci[oó]n\s+', '', texto).strip()

    # Si después de limpiar queda vacío, devolver None
    if not texto:
        return None

    # Determinar si es nacional o extranjero
    if 'España' in texto or 'spain' in texto.lower():
        return 'Nacional'
    else:
        return 'Extranjero'

## PASO 5.4: Función para filtrar comentarios válidos

In [ ]:
def filtrar_comentarios_validos(df):
    """
    Filtra comentarios que estén vacíos o sean muy cortos
    Elimina registros sin comentario válido o con menos de 5 palabras
    """

    print("🔍 Filtrando comentarios válidos...")
    print(f"Registros iniciales: {len(df)}")

    # Eliminar filas donde el comentario sea NaN o vacío
    df_filtrado = df[df['Comentario'].notna()].copy()
    df_filtrado = df_filtrado[df_filtrado['Comentario'].str.strip() != ''].copy()
    print(f"Tras eliminar comentarios vacíos: {len(df_filtrado)}")

    # Eliminar comentarios con menos de 5 palabras
    df_filtrado['num_palabras'] = df_filtrado['Comentario'].str.split().str.len()
    df_filtrado = df_filtrado[df_filtrado['num_palabras'] >= 5].copy()
    print(f"Tras eliminar comentarios con menos de 5 palabras: {len(df_filtrado)}")

    # Eliminar la columna auxiliar
    df_filtrado = df_filtrado.drop('num_palabras', axis=1)

    return df_filtrado

## PASO 5.5: Función para limpieza inicial del texto

In [ ]:
def limpiar_texto_preprocesamiento(texto):
    """
    Realiza la limpieza inicial del texto de comentarios de hoteles
    Elimina URLs, números y caracteres especiales
    """

    if not isinstance(texto, str):
        return ""

    # Convertir a minúsculas y eliminar URLs
    texto = texto.lower()
    texto = re.sub(r'http\S+', ' ', texto)

    # Eliminar números y caracteres especiales
    texto = re.sub(r'\d+', ' ', texto)
    texto = re.sub(r'[^\w\s]', ' ', texto)

    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()

    return texto

## PASO 5.6: Función para procesamiento avanzado con spaCy

In [ ]:
def procesar_texto_spacy(texto):
    """
    Procesa el texto usando spaCy con filtros mejorados para BERTopic
    """

    # Procesar el texto con el modelo de spaCy
    doc = nlp(texto)

    # Stopwords adicionales después de lematización
    stopwords_lemas = {"cabo", "poder", "hacer", "sido", "haber", "tener", "él", "día", "gran", "the", "decir", "dar",
                       "cada", "siempre", "traer", "volver", "tambien", "llegar", "incluso", "fin", "además", "vez"}

    # Filtrar y procesar cada token
    tokens_procesados = [
        token.lemma_
        for token in doc
        if token.text.lower() not in stop_words       # Stopwords originales
        and token.lemma_.lower() not in stopwords_lemas  # Stopwords de lemas
        and len(token.text) > 2
        and not token.is_punct
        and not token.is_space
        and token.is_alpha
        and not token.like_num               # Eliminar números
    ]

    return ' '.join(tokens_procesados)

## PASO 5.7: Aplicar todas las funciones de limpieza

In [ ]:
# Filtrar comentarios válidos
df = filtrar_comentarios_validos(df)

# Aplicar funciones de limpieza
try:
    df['calificacion_numerica'] = df.apply(extraer_calificacion, axis=1)
except:
    print("⚠️ Error en calificaciones, usando valores NaN")
    df['calificacion_numerica'] = np.nan

df['fecha_limpia'] = df['Fecha_estancia'].apply(convertir_fecha)
df['categoria_nacionalidad'] = df['Nacionalidad_contrib'].apply(limpiar_nacionalidad)

# Aplicar funciones de limpieza de texto
df['comentario_limpio'] = df['Comentario'].apply(limpiar_texto_preprocesamiento)
df['comentario_procesado'] = df['comentario_limpio'].apply(procesar_texto_spacy)

# Eliminar palabras con 3 o menos caracteres
df['comentario_procesado'] = df['comentario_procesado'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

# Calcular métricas adicionales del texto
df['longitud_comentario'] = df['Comentario'].fillna('').apply(len)
df['num_palabras'] = df['comentario_procesado'].str.split().str.len()

print(f"✅ Datos procesados: {len(df)} registros")

In [ ]:
# Crear nuevo DataFrame con solo las columnas relevantes
columnas_finales = ['Comentario', 'Nombre_hotel', 'calificacion_numerica', 'fecha_limpia', 'categoria_nacionalidad',
                    'comentario_limpio', 'comentario_procesado', 'longitud_comentario', 'num_palabras']

df_final = df[columnas_finales].copy()
print(f"🎯 DataFrame final creado con {len(df_final)} registros y {len(df_final.columns)} columnas")

## PASO 5.8: Verificar resultados del preprocesamiento

In [ ]:
# Mostrar ejemplo de procesamiento de texto
print("\n✏️ Ejemplo de procesamiento de texto:")
ejemplo_idx = 0
print("COMENTARIO ORIGINAL:")
print(df_final['Comentario'].iloc[ejemplo_idx])
print("\nCOMENTARIO LIMPIO:")
print(df_final['comentario_limpio'].iloc[ejemplo_idx])
print("\nCOMENTARIO PROCESADO:")
print(df_final['comentario_procesado'].iloc[ejemplo_idx])

# Mostrar las primeras 3 filas
display(df_final.head(3))

# PASO 6: ANÁLISIS EXPLORATORIO BÁSICO

## PASO 6.1: Estadísticas generales

In [ ]:
print("\n=== ESTADÍSTICAS GENERALES ===")
print(f"Total de reseñas analizadas: {len(df_final)}")
print(f"Calificación promedio: {df_final['calificacion_numerica'].mean():.2f} estrellas")
print(f"Calificación mediana: {df_final['calificacion_numerica'].median():.1f} estrellas")
print(f"Desviación estándar: {df_final['calificacion_numerica'].std():.2f}")
print(f"Longitud promedio de comentarios: {df_final['longitud_comentario'].mean():.0f} caracteres")
print(f"Número promedio de palabras: {df_final['num_palabras'].mean():.1f} palabras")

## PASO 6.2: Distribución de calificaciones

In [ ]:
# Paleta de colores accesible para daltónicos
# Gradiente de calificaciones: malo -> excelente
RATING_COLORS = ['#CC79A7', '#E69F00', '#F0E442', '#56B4E9', '#009E73']  # Rosa -> Naranja -> Amarillo -> Azul -> Verde

# Configuración tipográfica
plt.rcParams.update({'font.family': 'DejaVu Sans', 'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12, 'xtick.labelsize': 11, 'ytick.labelsize': 10,
    'figure.facecolor': 'white', 'axes.facecolor': 'white', 'axes.grid': True, 'grid.alpha': 0.3, 'grid.color': '#CCCCCC', 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.edgecolor': 'black'})

# ========== CREAR FIGURA ==========
plt.figure(figsize=(10, 6))

counts = df_final['calificacion_numerica'].value_counts().sort_index()

# Crear gráfico de barras
bars = plt.bar(counts.index, counts.values,
               color=RATING_COLORS[:len(counts)],
               alpha=0.85, edgecolor='black', linewidth=0.5)

# Títulos y etiquetas
plt.title('Distribución de Calificaciones', fontsize=14, pad=20)
plt.xlabel('Calificación (Estrellas)', fontsize=12)
plt.ylabel('Número de Reseñas', fontsize=12)

# Configurar ejes
plt.xticks([1, 2, 3, 4, 5], fontsize=11, color='#2c3e50')
plt.yticks(fontsize=10, color='#2c3e50')

# Añadir valores sobre las barras
for i, v in enumerate(counts.values):
    plt.text(counts.index[i], v + max(counts.values)*0.01, str(v),
             ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 6.3: Análisis temporal

In [ ]:
# Color principal para línea temporal
TEMPORAL_COLOR = '#009E73'  # Verde azulado

# Configuración tipográfica
plt.rcParams.update({'font.family': 'DejaVu Sans', 'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12, 'xtick.labelsize': 11, 'ytick.labelsize': 10,
    'figure.facecolor': 'white', 'axes.facecolor': 'white', 'axes.grid': True, 'grid.alpha': 0.3, 'grid.color': '#CCCCCC', 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.edgecolor': 'black'})

# ========== CREAR FIGURA ==========
plt.figure(figsize=(12, 6))

reseñas_por_mes = df_final.groupby('fecha_limpia').size().resample('M').sum()

# Crear gráfico de línea
plt.plot(reseñas_por_mes.index, reseñas_por_mes.values,
         marker='o', linewidth=2, markersize=6,
         color=TEMPORAL_COLOR, markerfacecolor=TEMPORAL_COLOR,
         markeredgecolor='black', markeredgewidth=0.5,
         alpha=0.85)

# Títulos y etiquetas
plt.title('Evolución Temporal de las Reseñas', fontsize=14, pad=20)
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Número de Reseñas', fontsize=12)

# Configurar ejes
plt.xticks(rotation=45, fontsize=11, color='#2c3e50')
plt.yticks(fontsize=10, color='#2c3e50')

plt.tight_layout()
plt.show()

## PASO 6.4: Distribución de nacionalidades

In [ ]:
# Colores para nacionalidades
NATIONALITY_COLORS = ['#009E73', '#E69F00']  # Verde azulado y naranja

# Configuración tipográfica
plt.rcParams.update({'font.family': 'DejaVu Sans', 'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12, 'xtick.labelsize': 11, 'ytick.labelsize': 10,
    'figure.facecolor': 'white', 'axes.facecolor': 'white', 'axes.grid': False, 'axes.spines.top': False, 'axes.spines.right': False, 'axes.spines.bottom': False,
    'axes.spines.left': False, 'axes.edgecolor': 'black'})

# ========== CREAR FIGURA ==========
plt.figure(figsize=(8, 6))

nac_counts = df_final['categoria_nacionalidad'].value_counts()

# Crear gráfico de pie con estilo
wedges, texts, autotexts = plt.pie(nac_counts.values,
                                  labels=nac_counts.index,
                                  autopct='%1.1f%%',
                                  colors=NATIONALITY_COLORS[:len(nac_counts)],
                                  startangle=90,
                                  wedgeprops=dict(edgecolor='black', linewidth=0.5),
                                  textprops={'fontsize': 11})

# Ajustar estilo de texto de porcentajes
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontsize(10)

# Título sin negrita
plt.title('Distribución de Nacionalidades de los Huéspedes',
          fontsize=14, pad=20)

plt.axis('equal')
plt.tight_layout()
plt.show()

## PASO 6.5: Hoteles más reseñados

In [ ]:
# Color principal para barras horizontales
HORIZONTAL_COLOR = '#56B4E9'  # Azul claro

# Configuración tipográfica
plt.rcParams.update({'font.family': 'DejaVu Sans', 'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12, 'xtick.labelsize': 11, 'ytick.labelsize': 10,
    'figure.facecolor': 'white', 'axes.facecolor': 'white', 'axes.grid': True, 'grid.alpha': 0.3, 'grid.color': '#CCCCCC', 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.edgecolor': 'black',})

# ========== CREAR FIGURA ==========
plt.figure(figsize=(12, 6))

hoteles_top = df_final['Nombre_hotel'].value_counts().head(10)

# Crear gráfico de barras horizontales
bars = plt.barh(range(len(hoteles_top)), hoteles_top.values,
                color=HORIZONTAL_COLOR, alpha=0.85,
                edgecolor='black', linewidth=0.5)

# Configurar etiquetas del eje Y
plt.yticks(range(len(hoteles_top)), hoteles_top.index,
           fontsize=10, color='#2c3e50')

# Títulos y etiquetas
plt.title('Top 10 Hoteles con Más Reseñas', fontsize=14, pad=20)
plt.xlabel('Número de Reseñas', fontsize=12)

# Configurar eje X
plt.xticks(fontsize=11, color='#2c3e50')

# Invertir eje Y para mostrar el mayor arriba
plt.gca().invert_yaxis()

# Añadir valores en las barras
for i, v in enumerate(hoteles_top.values):
    plt.text(v + max(hoteles_top.values)*0.01, i, str(v),
             va='center', ha='left', fontsize=10)

plt.tight_layout()
plt.show()

# PASO 7: ANÁLISIS DE SENTIMIENTOS

In [ ]:
# Inicializar el analizador de sentimientos VADER
sia = SentimentIntensityAnalyzer()

## PASO 7.1: Traducción de 100 comentarios al inglés para análisis con VADER

In [ ]:
# Para esta guía, trabajaremos con una muestra aleatoria pero reproducible de 100 comentarios
MUESTRA_SIZE = 100

df_muestra = df_final.sample(n=MUESTRA_SIZE, random_state=123).copy()

In [ ]:
# Función para traducir comentarios
def traducir_comentario(texto):
    """
    Traduce comentarios del español al inglés usando Google Translator

    Es necesario traducir porque VADER fue diseñado específicamente para inglés.
    Su diccionario léxico contiene palabras en inglés con intensidades emocionales,
    por lo que el análisis de sentimientos será mucho más preciso en inglés.
    """

    if pd.isna(texto) or texto.strip() == "":
        return ""

    try:
        translator = GoogleTranslator(source='es', target='en')
        texto_traducido = translator.translate(str(texto))
        time.sleep(0.1)  # Pausa para evitar límites de la API
        return texto_traducido
    except Exception as e:
        return str(texto)  # Devolver texto original si falla

In [ ]:
# Traducir la muestra de comentarios
comentarios_traducidos = []
total_comentarios = len(df_muestra)

for i, texto in enumerate(df_muestra['Comentario']):
    # Mostrar progreso cada 50 comentarios
    if i % 50 == 0:
        print(f"Progreso: {i+1}/{total_comentarios} ({((i+1)/total_comentarios*100):.1f}%)")

    comentario_traducido = traducir_comentario(texto)
    comentarios_traducidos.append(comentario_traducido)

# Agregar columna de comentarios traducidos
df_muestra['comentario_ingles'] = comentarios_traducidos

print(f"\n✅ Traducción completada: {len(df_muestra)} comentarios traducidos")
print("🎯 Muestra lista para análisis de sentimientos con VADER")

# Guardar la muestra para continuar el análisis
df_analisis = df_muestra.copy()

## PASO 7.2: Función para analizar sentimiento con VADER

In [ ]:
def analizar_sentimiento_vader(texto):
    """
    Analiza el sentimiento del texto usando VADER
    VADER devuelve puntuaciones para neg, neu, pos y compound
    - compound: puntuación general (-1 muy negativo a +1 muy positivo)
    """

    if pd.isna(texto) or texto == "":
        return {'compound': 0, 'pos': 0, 'neu': 1, 'neg': 0}

    # VADER analiza el texto traducido en inglés
    scores = sia.polarity_scores(str(texto))
    return scores

## PASO 7.3: Aplicar VADER a los comentarios traducidos

In [ ]:
sentimientos_vader = df_analisis['comentario_ingles'].apply(analizar_sentimiento_vader)

# Extraer las puntuaciones en columnas separadas
df_analisis['vader_compound'] = [s['compound'] for s in sentimientos_vader]
df_analisis['vader_positivo'] = [s['pos'] for s in sentimientos_vader]
df_analisis['vader_neutral'] = [s['neu'] for s in sentimientos_vader]
df_analisis['vader_negativo'] = [s['neg'] for s in sentimientos_vader]


## PASO 7.4: Categorizar sentimientos basado en compound score

In [ ]:
def categorizar_sentimiento_vader(compound_score):
    """
    Categoriza el sentimiento basado en el compound score de VADER
    - Positivo: compound >= 0.05
    - Neutral: -0.05 < compound < 0.05
    - Negativo: compound <= -0.05
    """

    if compound_score >= 0.05:
        return 'Positivo'
    elif compound_score <= -0.05:
        return 'Negativo'
    else:
        return 'Neutral'

df_analisis['sentimiento_vader'] = df_analisis['vader_compound'].apply(categorizar_sentimiento_vader)

## PASO 7.5: Estadísticas básicas deL análisis VADER

In [ ]:
print(f"\n📊 Resultados del análisis VADER:")
print(f"Puntuación promedio: {df_analisis['vader_compound'].mean():.3f}")
print(f"Rango: {df_analisis['vader_compound'].min():.3f} a {df_analisis['vader_compound'].max():.3f}")

sentimiento_counts = df_analisis['sentimiento_vader'].value_counts()
for sentimiento, cantidad in sentimiento_counts.items():
    porcentaje = (cantidad / len(df_analisis)) * 100
    print(f"{sentimiento}: {cantidad} reseñas ({porcentaje:.1f}%)")

## PASO 7.6: Ejemplos específicos de análisis VADER

In [ ]:
# Mostrar ejemplos de cada tipo de sentimiento
for sentimiento in ['Positivo', 'Neutral', 'Negativo']:
    ejemplos = df_analisis[df_analisis['sentimiento_vader'] == sentimiento].head(2)

    if len(ejemplos) > 0:
        print(f"\n🔍 Ejemplos de sentimiento {sentimiento.upper()}:")
        for idx, row in ejemplos.iterrows():
            print(f"Calificación: {row['calificacion_numerica']} estrellas")
            print(f"VADER Score: {row['vader_compound']:.3f}")
            print(f"Español: {row['comentario_limpio'][:100]}...")
            print(f"Inglés: {row['comentario_ingles'][:100]}...")
            print("-" * 50)

print("✅ Análisis de ejemplos completado")

## PASO 7.7: Visualización de resultados VADER

In [ ]:
# Paleta de colores accesible para daltónicos
colors_sentiment = {
    'Positivo': '#009E73',    # Verde azulado
    'Neutral': '#E69F00',     # Naranja
    'Negativo': '#CC79A7'     # Rosa magenta
}

# Configuración tipográfica
plt.rcParams.update({'font.family': 'DejaVu Sans', 'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12, 'xtick.labelsize': 11, 'ytick.labelsize': 10,
    'figure.facecolor': 'white', 'axes.facecolor': 'white', 'axes.grid': True, 'grid.alpha': 0.3, 'grid.color': '#CCCCCC', 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.edgecolor': 'black', 'savefig.dpi': 300})

# ========== CREAR FIGURA ==========
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1], hspace=0.3, wspace=0.3)

# ========== GRÁFICO 1: DISTRIBUCIÓN DE SENTIMIENTOS ==========
ax1 = fig.add_subplot(gs[0, 0])
colors_bars = [colors_sentiment[sent] for sent in sentimiento_counts.index]
bars = ax1.bar(sentimiento_counts.index, sentimiento_counts.values,
               color=colors_bars, alpha=0.85, edgecolor='black', linewidth=0.5)

# Añadir etiquetas
for bar, value in zip(bars, sentimiento_counts.values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + max(sentimiento_counts.values)*0.02,
             f'{value}',
             ha='center', va='bottom', fontsize=10)

ax1.set_title('Distribución de Sentimientos (VADER)', fontsize=14, pad=20)
ax1.text(-0.12, 1.05, 'a', transform=ax1.transAxes, fontsize=16,
         verticalalignment='top', horizontalalignment='left')
ax1.set_ylabel('Número de Reseñas', fontsize=12)
ax1.tick_params(axis='x', labelsize=11, colors='#2c3e50')
ax1.tick_params(axis='y', labelsize=10, colors='#2c3e50')

# ========== GRÁFICO 2: HISTOGRAMA ==========
ax2 = fig.add_subplot(gs[0, 1])
n, bins, patches = ax2.hist(df_analisis['vader_compound'], bins=30,
                           alpha=0.7, edgecolor='black', linewidth=0.5)

# Colorear barras según valor
for i, patch in enumerate(patches):
    bin_center = (bins[i] + bins[i+1]) / 2
    if bin_center >= 0.05:
        patch.set_facecolor(colors_sentiment['Positivo'])
    elif bin_center <= -0.05:
        patch.set_facecolor(colors_sentiment['Negativo'])
    else:
        patch.set_facecolor(colors_sentiment['Neutral'])

ax2.set_title('Distribución de Puntuaciones VADER', fontsize=14, pad=20)
ax2.text(-0.12, 1.05, 'b', transform=ax2.transAxes, fontsize=16,
         verticalalignment='top', horizontalalignment='left')
ax2.set_xlabel('VADER Compound Score', fontsize=12)
ax2.set_ylabel('Frecuencia', fontsize=12)
ax2.tick_params(axis='x', labelsize=11, colors='#2c3e50')
ax2.tick_params(axis='y', labelsize=10, colors='#2c3e50')

# ========== GRÁFICO 3: HEATMAP CON COLORES PERSONALIZADOS ==========
ax3 = fig.add_subplot(gs[1, :])
df_comparacion = pd.crosstab(df_analisis['calificacion_numerica'],
                            df_analisis['sentimiento_vader'],
                            normalize='index') * 100

# Crear gradiente personalizado
colors_list = ['#CC79A7', '#E69F00', '#009E73']  # Negativo -> Neutral -> Positivo
n_bins = 100
cmap_custom = mcolors.LinearSegmentedColormap.from_list('custom', colors_list, N=n_bins)

# Heatmap
sns.heatmap(df_comparacion.T, annot=True, fmt='.1f', cmap=cmap_custom,
            cbar_kws={'label': 'Porcentaje (%)'}, ax=ax3,
            linewidths=0)

ax3.set_title('Mapa de Calor: Sentimientos VADER vs Calificaciones',
              fontsize=14, pad=20)
ax3.text(-0.08, 1.05, 'c', transform=ax3.transAxes, fontsize=16,
         verticalalignment='top', horizontalalignment='left')
ax3.set_xlabel('Calificación (Estrellas)', fontsize=12)
ax3.set_ylabel('Sentimiento VADER', fontsize=12)
ax3.tick_params(rotation=0)

# ========== TÍTULO GENERAL ==========
plt.suptitle('Análisis Completo de Sentimientos con VADER',
             fontsize=16, y=0.98)

plt.tight_layout()
plt.show()

# PASO 8: CLASIFICACIÓN DE TEXTO

In [ ]:
# Inicializar clasificador BERT español
clasificador = pipeline("zero-shot-classification",
                        model="Recognai/bert-base-spanish-wwm-cased-xnli")

# Definir categorías para clasificar reseñas de hoteles
categorias = [
    "Ubicación y acceso",
    "Habitaciones e instalaciones",
    "Atención del personal",
    "Hotel limpio",
    "Precio",
    "Comidas y bebidas",
    "Opinión general"
]

## PASO 8.1: Ejemplos detallados con 5 comentarios

In [ ]:
muestra_ejemplos = df_analisis.head(5)

for i, row in muestra_ejemplos.iterrows():
    comentario = row['Comentario']

    print(f"--- Ejemplo {i+1} ---")
    print(f"📝 TEXTO COMPLETO:")

    # Dividir el comentario en líneas legibles
    palabras = comentario.split()
    lineas = []
    linea_actual = ""

    for palabra in palabras:
        if len(linea_actual + " " + palabra) <= 80:
            linea_actual += " " + palabra if linea_actual else palabra
        else:
            lineas.append(linea_actual)
            linea_actual = palabra

    if linea_actual:
        lineas.append(linea_actual)

    # Mostrar comentario formateado
    for linea in lineas:
        print(f"   {linea}")

    print()

    # Aplicar clasificación
    resultado = clasificador(comentario, categorias)

    # Mostrar resultados con barras visuales
    print(f"📊 Clasificación:")
    for j in range(len(categorias)):
        categoria = resultado['labels'][j]
        score = resultado['scores'][j]
        barra = "█" * int(score * 20)
        print(f"   {categoria:<25} {score:.3f} {barra}")

    print(f"\n✅ Categoría principal: {resultado['labels'][0]} ({resultado['scores'][0]:.3f})")
    print("="*70)
    print()

## PASO 8.2: Aplicar clasificación a 50 comentarios

In [ ]:
# Usar muestra aleatoria de 50 comentarios pero reproducible
muestra_analisis = df_final.sample(n=50, random_state=123)

categorias_principales = []
puntuaciones_maximas = []

for _, row in tqdm(muestra_analisis.iterrows(), total=len(muestra_analisis), desc="Clasificando comentarios"):
    comentario = row['Comentario']
    resultado = clasificador(comentario, categorias)

    # Guardar la categoría con mayor puntuación
    categorias_principales.append(resultado['labels'][0])
    puntuaciones_maximas.append(resultado['scores'][0])

# Crear DataFrame con resultados
df_resultados = pd.DataFrame({
    'categoria': categorias_principales,
    'confianza': puntuaciones_maximas
})

## PASO 8.3: Crear visualización de resultados

In [ ]:
# Paleta de colores accesible para daltónicos
NATURE_COLORS = ['#009E73', '#E69F00', '#CC79A7', '#56B4E9', '#F0E442', '#0072B2', '#D55E00']

# Configuración tipográfica
plt.rcParams.update({'font.family': 'DejaVu Sans', 'font.size': 11, 'axes.titlesize': 14, 'axes.labelsize': 12, 'xtick.labelsize': 11, 'ytick.labelsize': 10,
    'figure.facecolor': 'white', 'axes.facecolor': 'white', 'axes.grid': True, 'grid.alpha': 0.3, 'grid.color': '#CCCCCC', 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.edgecolor': 'black', 'savefig.dpi': 300})

# ========== CREAR FIGURA ==========
plt.figure(figsize=(12, 7))

conteo_categorias = df_resultados['categoria'].value_counts()

# Crear gráfico de barras
bars = plt.bar(range(len(conteo_categorias)), conteo_categorias.values,
               color=NATURE_COLORS[:len(conteo_categorias)],
               alpha=0.85, edgecolor='black', linewidth=0.5)

# Añadir valores sobre las barras
for i, (bar, value) in enumerate(zip(bars, conteo_categorias.values)):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + max(conteo_categorias.values)*0.01,
             f'{value}',
             ha='center', va='bottom', fontsize=10)

# Títulos y etiquetas
plt.title('Distribución de Categorías en Reseñas de Hoteles',
          fontsize=14, pad=20)
plt.xlabel('Categorías', fontsize=12)
plt.ylabel('Número de Reseñas', fontsize=12)

# Configurar etiquetas del eje x
plt.xticks(range(len(conteo_categorias)), conteo_categorias.index,
           rotation=45, ha='right', fontsize=11, color='#2c3e50')
plt.yticks(fontsize=10, color='#2c3e50')

# Panel label en esquina superior izquierda
plt.text(-0.08, 1.05, 'a', transform=plt.gca().transAxes, fontsize=16,
         verticalalignment='top', horizontalalignment='left')

plt.tight_layout()
plt.show()

# PASO 9: MODELADO DE TEMAS CON BERTOPIC

## PASO 9.1: Preparar datos para BERTopic

In [ ]:
# Usar muestra aleatoria pero reproducible de 2000 comentarios
muestra_bertopic = df_final.sample(n=2000, random_state=123)

# Convertir comentarios procesados a lista
comentarios = muestra_bertopic["comentario_procesado"].tolist()
timestamps = muestra_bertopic["fecha_limpia"].tolist()

## PASO 9.2: Configurar modelo BERTopic

In [ ]:
# Configurar UMAP para reducir dimensionalidad
umap_model = UMAP(
    n_neighbors=15,     # Número de vecinos para construir el grafo de similitudes
    n_components=5,     # Dimensiones finales del espacio reducido
    min_dist=0.0,       # Permite agrupación más estrecha de puntos
    metric='cosine',    # Distancia del coseno para embeddings de texto
    random_state=42     # Semilla para resultados reproducibles
)

# Configurar clustering con número específico de temas
cluster_model = KMeans(n_clusters=8, random_state=42)  # 8 temas para reseñas de hoteles

# Crear modelo BERTopic (sin número fijo de temas)
topic_model = BERTopic(
    language="spanish",         # Especificar idioma español
    verbose=True,              # Mostrar progreso
    umap_model=umap_model,      # Modelo de reducción de dimensionalidad
    hdbscan_model=cluster_model # Modelo de clustering
)

## PASO 9.3: Entrenar modelo

In [ ]:
# Ajustar modelo con los comentarios procesados
topics, probs = topic_model.fit_transform(comentarios)

## PASO 9.4: Mostrar información de temas

In [ ]:
# Obtener información general de temas
topic_info = topic_model.get_topic_info()
display(topic_info)

## PASO 9.5: Generar visualizaciones de BERTopic

In [ ]:
# Visualización 1: Mapa de temas en 2D
print("🗺️ Generando mapa de temas...")
fig_topics = topic_model.visualize_topics()
fig_topics.show()

# Visualización 2: Palabras clave por tema
print("📊 Generando gráfico de palabras clave...")
fig_barchart = topic_model.visualize_barchart(top_n_topics=8,n_words=9)
fig_barchart.show()

# Visualización 3: Jerarquía de temas
print("🌳 Generando jerarquía de temas...")
fig_hierarchy = topic_model.visualize_hierarchy(width=800, height=600)
fig_hierarchy.show()

# Visualización 4: Evolución temporal de temas
print("⏰ Analizando evolución temporal...")
topics_over_time = topic_model.topics_over_time(
    docs=comentarios,
    timestamps=timestamps,
    global_tuning=True,
    evolution_tuning=True,
    nr_bins=10  # 10 períodos temporales
)

fig_time = topic_model.visualize_topics_over_time(
    topics_over_time
)
fig_time.show()